In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as stats

In [2]:
afri= pd.read_csv("acled_conflict_data.csv")
afri.head()


,country,date,non_violence,violence
0,Zimbabwe,2019-03-01,0,2
1,Zimbabwe,2019-02-01,4,17
2,Zimbabwe,2019-01-01,27,19
3,Zimbabwe,2018-12-01,5,3
4,Zimbabwe,2018-11-01,4,3


In [3]:
#let's first clean the year column
afri["date"]=afri["date"].astype(str) #converting data type to string for further processing
afri["year"]= afri["date"]
l= list(f"{k}" for k in range(0,10))
for i in range(len(afri["date"])): #converting date to year
    if afri.date[i][0] in l:
        afri["year"][i]=afri["date"][i][0:4]
    else:
        afri["year"][i]=afri["date"][i][-4:]
afri.drop(columns="date",inplace=True) 
afri.head()

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,country,non_violence,violence,year
0,Zimbabwe,0,2,2019
1,Zimbabwe,4,17,2019
2,Zimbabwe,27,19,2019
3,Zimbabwe,5,3,2018
4,Zimbabwe,4,3,2018


In [4]:
afri=afri[["country","year","non_violence","violence"]] #changing column order
#afri=afri.query('year!="2019"') this also works
afri=afri[~afri["year"].isin(["2019"])]
afri.index=range(len(afri))
afri.head()

,country,year,non_violence,violence
0,Zimbabwe,2018,5,3
1,Zimbabwe,2018,4,3
2,Zimbabwe,2018,6,5
3,Zimbabwe,2018,4,2
4,Country Name = *ZIMBABWE*,2018,17,46


In [5]:
# Then let's clean the country column.
for i in range(len(afri["country"])):
    if "*" in afri["country"][i]:
        location=afri["country"][i].index("*")
        afri["country"][i]=afri["country"][i][location+1:-1]
    else:
        pass

afri["country"]=afri["country"].str.lower()
afri.country.unique()

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


array(['zimbabwe', 'zambia', 'uganda', 'tunisia', 'togo', 'tanzania',
       'sudan', 'south sudan', 'south africa', 'somalia', 'sierra leone',
       'senegal', 'rwanda', 'republic of congo', 'nigeria', 'niger',
       'namibia', 'mozambique', 'morocco', 'mauritania', 'mali', 'malawi',
       'madagascar', 'libya', 'liberia', 'lesotho', 'kenya',
       'ivory coast', 'guinea-bissau', 'guinea', 'ghana', 'gambia',
       'gabon', 'ethiopia', 'eswatini', 'eritrea', 'equatorial guinea',
       'egypt', 'djibouti', 'democratic republic of congo', 'chad',
       'central african republic', 'cameroon', 'burundi', 'burkina faso',
       'botswana', 'benin', 'angola', 'algeria'], dtype=object)

In [6]:
afri=afri.groupby(["country","year"]).sum().reset_index()
#Finally we are done cleaning the first dataframe!

In [7]:
#Rename the column names for merging in the future
afri=afri.rename(columns={"country":"Country Name","year":"Year"},)
afri.head()

,Country Name,Year,non_violence,violence
0,algeria,1997,0,130
1,algeria,1998,1,57
2,algeria,1999,0,56
3,algeria,2000,2,165
4,algeria,2001,109,142


In [8]:
gdp=pd.read_csv("API_NY.GDP.MKTP.KD.ZG_DS2_en_csv_v2_422196.csv",skiprows = 4)
gdp

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.446055,-1.369863,4.198232,0.848228,-0.450586,-0.211225,1.330511,NaN,NaN,NaN
1,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.426355,12.752287,5.600745,2.724543,1.451315,2.260314,2.665292,1.030660,NaN,NaN
2,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.471981,8.542147,4.954590,4.822626,0.943576,-2.580097,-0.147207,-2.133493,NaN,NaN
3,Albania,ALB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,2.545405,1.417526,1.001988,1.770000,2.230000,3.350000,3.836620,4.004413,NaN,NaN
4,Andorra,AND,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.646543,-1.615218,0.351645,2.277683,0.842204,1.889124,1.724022,1.629345,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,4.812563,2.895349,3.431230,1.198071,4.094455,4.056579,4.227909,4.145372,NaN,NaN
260,"Yemen, Rep.",YEM,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-12.714897,2.392990,4.823519,-0.188690,-16.678463,-13.621458,-5.942320,-2.701475,NaN,NaN
261,South Africa,ZAF,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,3.844751,6.177883,7.373613,7.939782,6.122761,...,3.284168,2.213355,2.485201,1.846992,1.193733,0.399088,1.414513,0.787056,NaN,NaN
262,Zambia,ZMB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,1.361382,-2.490839,3.272393,12.214048,16.647456,...,5.564625,7.597617,5.059376,4.695826,2.919881,3.757178,3.403169,3.794901,NaN,NaN


In [9]:
#drop the columns we don't need
gdp.drop(columns=["Country Code","Indicator Code","2019","Unnamed: 64","Indicator Name"],inplace=True)
gdp.drop(columns=[f"{i}" for i in range(1960,1997)],axis=1,inplace=True)
gdp.head()

,Country Name,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,7.046872,1.991988,1.238039,7.616590,-2.971257,-3.273647,1.975547,7.911565,1.214349,...,-10.519748,-3.685030,3.446055,-1.369863,4.198232,0.848228,-0.450586,-0.211225,1.330511,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,8.832278,1.414118,11.229715,...,21.390528,14.362441,0.426355,12.752287,5.600745,2.724543,1.451315,2.260314,2.665292,1.030660
2,Angola,7.274277,4.691146,2.181490,3.054624,4.205999,13.665687,2.989850,10.952862,15.028915,...,0.858713,4.859220,3.471981,8.542147,4.954590,4.822626,0.943576,-2.580097,-0.147207,-2.133493
3,Albania,-10.919984,8.830088,12.889897,6.950036,8.290070,4.539961,5.530051,5.509999,5.529915,...,3.349994,3.706881,2.545405,1.417526,1.001988,1.770000,2.230000,3.350000,3.836620,4.004413
4,Andorra,9.067672,3.194790,4.099081,3.528361,4.546768,6.471015,12.168720,7.647870,7.396983,...,-3.690654,-5.358826,-4.646543,-1.615218,0.351645,2.277683,0.842204,1.889124,1.724022,1.629345


In [10]:
#drop the observations that is not from Africa
gdp["Country Name"]=gdp["Country Name"].str.lower()
for i in range(len(gdp["Country Name"])):
    if gdp["Country Name"][i] in afri["Country Name"].unique():
        pass
    else:
        gdp.drop([i],inplace=True)

In [11]:
gdp.index=range(len(gdp))
gdp.head()

,Country Name,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,angola,7.274277,4.691146,2.181490,3.054624,4.205999,13.665687,2.989850,10.952862,15.028915,...,0.858713,4.859220,3.471981,8.542147,4.954590,4.822626,0.943576,-2.580097,-0.147207,-2.133493
1,burundi,-1.590000,4.750000,-1.010000,-0.856864,2.055807,4.446519,-1.223728,4.833658,0.900000,...,3.812747,5.124163,4.032602,4.446708,4.924195,4.240652,-3.900003,-0.600020,0.500010,1.599977
2,benin,5.734688,3.961012,5.341449,5.859992,5.330411,4.644470,3.444045,4.429629,1.711578,...,2.329301,2.110199,2.962750,4.816478,7.189716,6.351832,2.095808,3.964860,5.837823,6.858002
3,burkina faso,6.316835,7.307720,7.404179,1.820242,6.613400,4.353004,7.802440,4.478474,8.661862,...,2.962014,5.373373,6.626865,6.452699,5.792589,4.326837,3.894548,5.934135,6.302096,6.509803
4,botswana,8.325891,0.443663,9.667241,1.987696,0.250574,6.069531,4.625895,2.705822,4.556646,...,-7.652310,8.563632,6.048316,4.456167,11.343424,4.149290,-1.697966,4.303773,2.907535,4.453560


In [12]:
#unpivot year variable
gdp=gdp.melt(id_vars="Country Name",var_name ="Year",value_name="GDP growth rate")
gdp

,Country Name,Year,GDP growth rate
0,angola,1997,7.274277
1,burundi,1997,-1.590000
2,benin,1997,5.734688
3,burkina faso,1997,6.316835
4,botswana,1997,8.325891
...,...,...,...
963,tanzania,2018,5.200000
964,uganda,2018,6.090145
965,south africa,2018,0.787056
966,zambia,2018,3.794901


In [13]:
for i in range(len(gdp)):
    gdp.iloc[i,].fillna(gdp.iloc[i,2:].mean(),inplace=True)
    
gdp.head(10)

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Country Name,Year,GDP growth rate
0,angola,1997,7.274277
1,burundi,1997,-1.590000
2,benin,1997,5.734688
3,burkina faso,1997,6.316835
4,botswana,1997,8.325891
5,central african republic,1997,5.299936
6,cameroon,1997,5.231321
7,djibouti,1997,NaN
8,algeria,1997,1.100000
9,eritrea,1997,7.908685


In [14]:
workers = pd.read_csv("API_SL.TLF.TOTL.IN_DS2_en_csv_v2_423580.csv", skiprows = 4)
workers.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,10008297.0,10534726.0,11128394.0,11752833.0,12381396.0,12931832.0,13467695.0,13952821.0,14450224.0,NaN
2,Angola,AGO,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,9968105.0,10327972.0,10700189.0,11084705.0,11481285.0,11882159.0,12295573.0,12728920.0,13183538.0,NaN
3,Albania,ALB,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,1271089.0,1322878.0,1244852.0,1268612.0,1322597.0,1321380.0,1323346.0,1322907.0,1324517.0,NaN
4,Andorra,AND,"Labor force, total",SL.TLF.TOTL.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#drop the columns we don't need
workers.drop(columns=["Country Code","Indicator Code","2019","Unnamed: 64","Indicator Name"],inplace=True)
workers.drop(columns=[f"{i}" for i in range(1960,1997)],axis=1,inplace=True)
workers.head(3)

,Country Name,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,6318137.0,6408475.0,6525062.0,6701390.0,7000745.0,7379318.0,7801160.0,8219904.0,8607338.0,...,9312008.0,9572580.0,10008297.0,10534726.0,11128394.0,11752833.0,12381396.0,12931832.0,13467695.0,13952821.0
2,Angola,6064208.0,6272599.0,6494291.0,6728180.0,6958416.0,7206353.0,7464872.0,7737243.0,8022529.0,...,9272188.0,9623400.0,9968105.0,10327972.0,10700189.0,11084705.0,11481285.0,11882159.0,12295573.0,12728920.0


In [16]:
#drop the observations that is not from Africa
workers["Country Name"]=workers["Country Name"].str.lower()
for i in range(len(workers["Country Name"])):
    if workers["Country Name"][i] in afri["Country Name"].unique():
        pass
    else:
        workers.drop([i],inplace=True)

#reset index
workers.index=range(len(workers))
#unpivot year variable
workers=workers.melt(id_vars="Country Name",var_name ="Year",value_name="Labor Force Total")
workers.head()

,Country Name,Year,Labor Force Total
0,angola,1997,6064208.0
1,burundi,1997,2608184.0
2,benin,1997,2488738.0
3,burkina faso,1997,4496091.0
4,botswana,1997,553963.0


In [17]:
len(workers["Country Name"])

968

In [18]:
popu = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_422125.csv", skiprows = 4)
popu.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN,NaN


In [19]:
#drop the columns we don't need
popu.drop(columns=["Country Code","Indicator Code","2019","Unnamed: 64","Indicator Name"],inplace=True)
popu.drop(columns=[f"{i}" for i in range(1960,1997)],axis=1,inplace=True)
popu.head(3)

#drop the observations that is not from Africa
popu["Country Name"]=popu["Country Name"].str.lower()
for i in range(len(popu["Country Name"])):
    if popu["Country Name"][i] in afri["Country Name"].unique():
        pass
    else:
        popu.drop([i],inplace=True)

#reset index
popu.index=range(len(popu))
#unpivot year variable
popu=popu.melt(id_vars="Country Name",var_name ="Year",value_name="Total Population")
popu.head()

,Country Name,Year,Total Population
0,angola,1997,14871570.0
1,burundi,1997,6122130.0
2,benin,1997,6281639.0
3,burkina faso,1997,10665546.0
4,botswana,1997,1540425.0


In [20]:
#let's merge the data we cleaned!
final=pd.merge(left=afri,right=gdp,how="left")
final=pd.merge(left=final,right=workers,how="left")
final=pd.merge(left=final,right=popu,how="left")
final.head(3)

,Country Name,Year,non_violence,violence,GDP growth rate,Labor Force Total,Total Population
0,algeria,1997,0,130,1.100000,8249066.0,29742979.0
1,algeria,1998,1,57,5.100004,8457585.0,30192754.0
2,algeria,1999,0,56,3.200002,8668123.0,30623406.0


In [21]:
final["GDP growth rate"].fillna(final.groupby("Country Name")["GDP growth rate"].mean(),inplace=True)
final["Labor Force Total"].fillna(final.groupby("Country Name")["Labor Force Total"].mean(),inplace=True)
final["Total Population"].fillna(final.groupby("Country Name")["Total Population"].mean(),inplace=True)
final.head()

,Country Name,Year,non_violence,violence,GDP growth rate,Labor Force Total,Total Population
0,algeria,1997,0,130,1.100000,8249066.0,29742979.0
1,algeria,1998,1,57,5.100004,8457585.0,30192754.0
2,algeria,1999,0,56,3.200002,8668123.0,30623406.0
3,algeria,2000,2,165,3.819678,8878121.0,31042235.0
4,algeria,2001,109,142,3.008395,9104651.0,31451514.0


In [22]:
final.dropna(inplace=True)

In [23]:
#test skewness
final.skew()

non_violence         5.838268
violence             3.809880
GDP growth rate      7.099966
Labor Force Total    2.864260
Total Population     3.492661
dtype: float64

In [24]:
print(final.violence.min())
print(final.non_violence.min())
# We need to add one to the variables that have 0 values to log them

0
0


In [25]:
final.violence=final.violence+1
final.non_violence=final.non_violence+1

In [26]:
# log the variables as they are all right skewed
final.violence=np.log(final.violence)
final.non_violence=np.log(final.non_violence)
final["GDP growth rate"]=np.log(final["GDP growth rate"]+1)
final["Labor Force Total"]=np.log(final["Labor Force Total"]+1)
final["Total Population"]=np.log(final["Total Population"]+1)
final.head()

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Country Name,Year,non_violence,violence,GDP growth rate,Labor Force Total,Total Population
0,algeria,1997,0.000000,4.875197,0.741937,15.925611,17.208104
1,algeria,1998,0.693147,4.060443,1.808289,15.950574,17.223113
2,algeria,1999,0.000000,4.043051,1.435085,15.975163,17.237275
3,algeria,2000,1.098612,5.111988,1.572707,15.999101,17.250859
4,algeria,2001,4.700480,4.962845,1.388391,16.024296,17.263958


In [27]:
final["Total Population"].unique()

array([17.2081037 , 17.22311255, 17.23727521, 17.25085929, 17.26395771,
       17.27670837, 17.28946751, 17.30264597, 17.31654498, 17.33125621,
       17.34677003, 17.36313196, 17.38035289, 17.39840308, 17.41723622,
       17.43675037, 17.45677765, 17.47711357, 17.49756744, 17.51808099,
       17.53853051, 17.55860425, 16.51496196, 16.54725137, 16.57974385,
       16.61251588, 16.64552786, 16.67882042, 16.71260853, 16.74713867,
       16.78251424, 16.81870998, 16.85551633, 16.89262165, 16.92966042,
       16.96637504, 17.00271663, 17.03869437, 17.07421387, 17.1091888 ,
       17.14357731, 17.17736003, 17.21058084, 17.24334218, 15.65314165,
       15.68272778, 15.71224532, 15.74208526, 15.77232306, 15.80275389,
       15.83315063, 15.86320405, 15.89272788, 15.92170332, 15.95024418,
       15.97847973, 16.00657274, 16.0346336 , 16.06267078, 16.09064727,
       16.11855445, 16.14637626, 16.1740934 , 16.20170749, 16.22920804,
       16.25655666, 14.24756956, 14.2702895 , 14.31223827, 14.36

In [28]:
y=final["GDP growth rate"]
x=final[["Labor Force Total","Total Population","non_violence","violence"]]
x=stats.add_constant(x)
reg=stats.OLS(y,x,missing = 'drop').fit()
reg.summary()

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        GDP growth rate   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     4.183
Date:                Mon, 25 Nov 2019   Prob (F-statistic):            0.00233
Time:                        17:33:54   Log-Likelihood:                -948.70
No. Observations:                 814   AIC:                             1907.
Df Residuals:                     809   BIC:                             1931.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.3195      0.502      0.637      0.525      -0.666       1.305
Labor Force Total     0.1375      0.172      0.798      0.425      -0.201       0.476
Total Population     -0.0388      0.181     -0.215      0.830      -0.394       0.317
non_violence         -0.0506      0.021     -2.358      0.019      -0.093      -0.008
violence             -0.0078      0.020     -0.386      0.700      -0.047       0.032
==============================================================================
Omnibus:                      432.317   Durbin-Watson:                   1.447
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5806.600
Skew:                          -2.088   Prob(JB):                         0.00
Kurtosis:                      15.400   Cond. No.                         436.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

 I googled around but still have no idea why my results are all nans. I already dropped all the NAN in my data and the number of observations is undoubtedly bigger that that of variables. If anyone have any clues on that please tell me and I will very much appreciate it. (｡･ω･)ﾉﾞ

## Xi's Revise:

Thanks to Emma's response to Amina's code,I found the problem of my nan results. That is because of: 1) negative values in variable producing new nan values in logged variables. 2)0 values in variable producing -inf values in logged variables. The OLS model works fine after correcting them.

Interpretation of the OLS model: It seems only non_violence variable is statistically significant. One percent increase in non_violence activity averagely result in 0.0005 decrease of GDP growth rate. Other variables do not seem to influence GDP growth.

Besides, the model fit is very low, which means we might need another model, probably a polynominal one, to have a better interpretation of the story.

In [29]:
np.log(0)

/Users/lixisherry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [30]:
# 2. another way to transform date into year
afri_new= pd.read_csv("acled_conflict_data.csv")
afri_new["date"]=afri_new["date"].astype("datetime64")
afri_new['year'] = afri_new['date'].dt.year
afri_new['year'].head()

0    2019
1    2019
2    2019
3    2018
4    2018
Name: year, dtype: int64

## Xiyang comment:

Good job! There are two things I want to mention:
1. there is a very useful function that can convert str to the date data type

In [31]:
afri_new= pd.read_csv("acled_conflict_data.csv")
date = afri_new.date  
afri_new['date'] = pd.to_datetime(date)  
afri_new['year'] = afri_new['date'].map(lambda x: 1*x.year)
afri_new.head()

,country,date,non_violence,violence,year
0,Zimbabwe,2019-03-01,0,2,2019
1,Zimbabwe,2019-02-01,4,17,2019
2,Zimbabwe,2019-01-01,27,19,2019
3,Zimbabwe,2018-12-01,5,3,2018
4,Zimbabwe,2018-11-01,4,3,2018


2. Python throw an error when it is running the code to drop the observations that is not from Africa. I guess it might be you put a wrong column name when define the data range.  

The code with a right column name is below:

In [32]:
workers["Country Name"]=workers["Country Name"].str.lower()
for i in range(len(workers["Country Name"])):
    if workers["Country Name"][i] in afri["Country Name"].unique():
        pass
    else:
        workers.drop([i],inplace=True)